# Predicting NBA Game Winners

In [1]:
# Load necessary libraries

import platform
import numpy as np
import pandas as pd
import sklearn as sk
import os

In [2]:
# Load the dataset: NBA statistics from the 2013-2014 basketball season
df = pd.read_csv('./datasets/NBA Regular Season Results 2013-2014.csv')

In [3]:
from sklearn.metrics import f1_score, make_scorer, classification_report

scorer = make_scorer(f1_score, pos_label = None, average = 'weighted')

In [4]:
# Examine the dataset with its initial formatting
df.head()

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,Tue Oct 29 2013,7:00 pm,Orlando Magic,87,Indiana Pacers,97,Box Score,NaN,18165,NaN
1,Tue Oct 29 2013,10:30 pm,Los Angeles Clippers,103,Los Angeles Lakers,116,Box Score,NaN,18997,NaN
2,Tue Oct 29 2013,8:00 pm,Chicago Bulls,95,Miami Heat,107,Box Score,NaN,19964,NaN
3,Wed Oct 30 2013,7:00 pm,Brooklyn Nets,94,Cleveland Cavaliers,98,Box Score,NaN,20562,NaN
4,Wed Oct 30 2013,8:30 pm,Atlanta Hawks,109,Dallas Mavericks,118,Box Score,NaN,19834,NaN


In [5]:
df.dtypes

# Note the Date is read as an object field

Date               object
Start (ET)         object
Visitor/Neutral    object
PTS                 int64
Home/Neutral       object
PTS.1               int64
Unnamed: 6         object
Unnamed: 7         object
Attend.             int64
Notes              object
dtype: object

In [6]:
# Don't read the first row because it is blank, and read the date column as a date

df = pd.read_csv('./datasets/NBA Regular Season Results 2013-2014.csv', parse_dates = ['Date'])

# Correct column names
df.columns = ['Date', 'Start Time (ET)', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 'Box Score', 
              'OT', 'Attendance', 'Notes']

In [7]:
df.shape

(1230, 10)

In [8]:
# View the updated dataset
df.head()

,Date,Start Time (ET),Visitor Team,Visitor Score,Home Team,Home Score,Box Score,OT,Attendance,Notes
0,2013-10-29,7:00 pm,Orlando Magic,87,Indiana Pacers,97,Box Score,NaN,18165,NaN
1,2013-10-29,10:30 pm,Los Angeles Clippers,103,Los Angeles Lakers,116,Box Score,NaN,18997,NaN
2,2013-10-29,8:00 pm,Chicago Bulls,95,Miami Heat,107,Box Score,NaN,19964,NaN
3,2013-10-30,7:00 pm,Brooklyn Nets,94,Cleveland Cavaliers,98,Box Score,NaN,20562,NaN
4,2013-10-30,8:30 pm,Atlanta Hawks,109,Dallas Mavericks,118,Box Score,NaN,19834,NaN


In [9]:
# The Date field has been updated to read as a datetime field
# May not do anything with the field yet
df.dtypes

Date               datetime64[ns]
Start Time (ET)            object
Visitor Team               object
Visitor Score               int64
Home Team                  object
Home Score                  int64
Box Score                  object
OT                         object
Attendance                  int64
Notes                      object
dtype: object

In [10]:
# Create new feature that notes whether the home team won
df['Home Win'] = df['Visitor Score'] < df['Home Score']

# Assign "class values" to the Home Win column

### This will also be the value that we want to predict ###
y_true = df['Home Win'].values

In [11]:
# Review new column with correctly populated fields
df.head()

,Date,Start Time (ET),Visitor Team,Visitor Score,Home Team,Home Score,Box Score,OT,Attendance,Notes,Home Win
0,2013-10-29,7:00 pm,Orlando Magic,87,Indiana Pacers,97,Box Score,NaN,18165,NaN,True
1,2013-10-29,10:30 pm,Los Angeles Clippers,103,Los Angeles Lakers,116,Box Score,NaN,18997,NaN,True
2,2013-10-29,8:00 pm,Chicago Bulls,95,Miami Heat,107,Box Score,NaN,19964,NaN,True
3,2013-10-30,7:00 pm,Brooklyn Nets,94,Cleveland Cavaliers,98,Box Score,NaN,20562,NaN,True
4,2013-10-30,8:30 pm,Atlanta Hawks,109,Dallas Mavericks,118,Box Score,NaN,19834,NaN,True


## What is the baseline?

How frequently do the home teams win? Typically, the assumption is that home teams will win more often than not

In [12]:
# Establishing the baseline win percentage for the Home Teams overall
n_games = df['Home Win'].count()
n_homewins = df['Home Win'].sum()
win_percentage = n_homewins / n_games

print('Home Win percentage: {0:.2f}%'.format(100 * win_percentage))

Home Win percentage: 58.05%


The home teams' win percentage is 58.0%. If a model is considered useful, it must (at minimum) have a better score than the established baseline

In [13]:
# Predicting the baseline for Home Teams wins with simple classifier

from sklearn.metrics import f1_score

y_pred = [1] * len(y_true)
print('F1: {0:.4f}%'.format(f1_score(y_true, y_pred, pos_label = None, average = 'weighted') * 100))

F1: 42.6408%


/Users/slic_vic/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


The calculated F1 score of 42.6% is the score to beat.

## Creating More Features

In [14]:
# Create two new columns, with all of the values set to False
df['Home Last Win'] = False
df['Visitor Last Win'] = False

In [15]:
# Determining whether the home and visitor teams won their last games
# This will update the Home Last Win & Visitor Last Win columns to either remain False or change to True

from collections import defaultdict

won_last = defaultdict(int)   # The default value of int is 0

# Inefficient method to deterime previous wins
for index, row in df.iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    row['Home Last Win'] = won_last[home_team]
    row['Visitor Last Win'] = won_last[visitor_team]
    # Set current win
    won_last[home_team] = row['Home Win']
    won_last[visitor_team] = not row['Home Win']
df.loc[20:25]

,Date,Start Time (ET),Visitor Team,Visitor Score,Home Team,Home Score,Box Score,OT,Attendance,Notes,Home Win,Home Last Win,Visitor Last Win
20,2013-11-01,7:30 pm,Milwaukee Bucks,105,Boston Celtics,98,Box Score,NaN,18624,NaN,False,False,False
21,2013-11-01,8:00 pm,Miami Heat,100,Brooklyn Nets,101,Box Score,NaN,17732,NaN,True,False,False
22,2013-11-01,7:00 pm,Cleveland Cavaliers,84,Charlotte Bobcats,90,Box Score,NaN,18017,NaN,True,False,False
23,2013-11-01,9:00 pm,Portland Trail Blazers,113,Denver Nuggets,98,Box Score,NaN,19155,NaN,False,False,False
24,2013-11-01,8:00 pm,Dallas Mavericks,105,Houston Rockets,113,Box Score,NaN,18142,NaN,True,False,False
25,2013-11-01,10:30 pm,San Antonio Spurs,91,Los Angeles Lakers,85,Box Score,NaN,18997,NaN,False,False,False


In [16]:
df.loc[90:95][['Date', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 
               'Home Win', 'Home Last Win', 'Visitor Last Win']]

,Date,Visitor Team,Visitor Score,Home Team,Home Score,Home Win,Home Last Win,Visitor Last Win
90,2013-11-09,Dallas Mavericks,91,Milwaukee Bucks,83,False,False,False
91,2013-11-09,Portland Trail Blazers,96,Sacramento Kings,85,False,False,False
92,2013-11-09,Utah Jazz,91,Toronto Raptors,115,True,False,False
93,2013-11-10,Minnesota Timberwolves,113,Los Angeles Lakers,90,False,False,False
94,2013-11-10,San Antonio Spurs,120,New York Knicks,89,False,False,False
95,2013-11-10,Washington Wizards,105,Oklahoma City Thunder,106,True,False,False


# Basic Classification with a Decision Tree

The default settings for the Decision Tree will be used for the home wins, winning streaks, and rankings

In [17]:
# Basic Decision Tree Classifier set up
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state = 14) # Remove random_state to get non-replicable results

In [18]:
from sklearn.cross_validation import cross_val_score

# Use selected features as input for the classifier (target)
X_previouswins = df[['Home Last Win', 'Visitor Last Win']].values

# Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_previouswins, y_true, scoring = scorer)

# Print results
print("Using just the last result from the home and visitor teams")
print('F1: {0:.4f}%'.format(np.mean(scores) * 100))

Using just the last result from the home and visitor teams
F1: 42.6408%


/Users/slic_vic/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/slic_vic/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/slic_vic/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/slic_vic/anaconda3/lib/python3.6/site-packages/sklearn/met

When using the features that deterime whether a team won their previous game, the result is 42.6%, which is equal to the baseline score

There was no improvement.


*** In the lecture, the result here was closer to 50-51%. What is happening here for the Decision Tree to result in the same score as the baseline? ***

### Decision Tree Classifier: Considerating Winning Streaks

In [19]:
# Taking into consideration winning streaks - What are the teams' win streaks coming into the game?
df["Home Win Streak"] = 0
df["Visitor Win Streak"] = 0

# Did the home and visitor teams win their last game?
from collections import defaultdict
win_streak = defaultdict(int)

for index, row in df.iterrows():  # Note that this is not the most efficient method
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["Home Win Streak"] = win_streak[home_team]
    row["Visitor Win Streak"] = win_streak[visitor_team]
    df.loc[index] = row    
    
    # Set current win streak
    if row["Home Win"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1

In [20]:
df.loc[50:60][['Date', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 
               'Home Win', 'Home Win Streak', 'Visitor Win Streak']]

,Date,Visitor Team,Visitor Score,Home Team,Home Score,Home Win,Home Win Streak,Visitor Win Streak
50,2013-11-05,Utah Jazz,88,Brooklyn Nets,104,True,0,0
51,2013-11-05,Los Angeles Lakers,104,Dallas Mavericks,123,True,1,1
52,2013-11-05,San Antonio Spurs,102,Denver Nuggets,94,False,0,0
53,2013-11-05,Indiana Pacers,99,Detroit Pistons,91,False,1,3
54,2013-11-05,Phoenix Suns,104,New Orleans Pelicans,98,False,1,0
55,2013-11-05,Charlotte Bobcats,102,New York Knicks,97,False,0,0
56,2013-11-05,Houston Rockets,116,Portland Trail Blazers,101,False,2,0
57,2013-11-05,Atlanta Hawks,105,Sacramento Kings,100,False,0,0
58,2013-11-05,Miami Heat,104,Toronto Raptors,95,False,1,1
59,2013-11-06,Utah Jazz,87,Boston Celtics,97,True,0,0


In [21]:
# Use selected features as input for the classifier (target)
X_winstreak = df[["Home Last Win", "Visitor Last Win", "Home Win Streak", "Visitor Win Streak"]].values

# Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_winstreak, y_true, scoring = scorer)

# Print results
print("Using whether the home team is ranked higher")
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
F1: 52.4982%


When taking into consideration whether a team is on a win streak, the model's results show 52.5%

### Decision Tree Classifier: Based on Previous Season Standings

In [22]:
# Identify which team is higher in the standings, based on the previous year's regular season final standings
# Load the standings data file

rank = pd.read_csv('./datasets/2012-2013 Regular Season Standings.csv', index_col = 'Team')

In [23]:
rank.head(3)

,Rk,Overall,Home,Road,E,W,A,C,SE,NW,...,Post,≤3,≥10,Oct,Nov,Dec,Jan,Feb,Mar,Apr
Team,,,,,,,,,,,,,,,,,,,,,
Miami Heat,1,66-16,37-4,29-12,41-11,25-5,14-4,12-6,15-1,8-2,...,30-2,9-3,39-8,1-0,10-3,10-5,8-5,12-1,17-1,8-1
Oklahoma City Thunder,2,60-22,34-7,26-15,21-9,39-13,7-3,8-2,6-4,10-6,...,21-8,3-6,44-6,NaN,13-4,11-2,11-5,7-4,12-5,6-2
San Antonio Spurs,3,58-24,35-6,23-18,25-5,33-19,8-2,9-1,8-2,9-9,...,16-12,9-5,31-10,1-0,12-4,12-4,12-3,8-3,10-4,3-6


In [24]:
# Rename columns in the rank DataFrame
rank.columns = ['Rank', 'Overall', 'Home', 'Road', 'Eastern Conference', 'Western Conference', 
                'Atlantic Div', 'Central Div', 'Southeast Div', 'Northwest Div', 'Pacific Div', 'Southwest Div', 
                'Pre All-Star', 'Post All-Star', 'Margin ≤3', 'Margin ≥10', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 
                'Mar', 'Apr']

In [25]:
rank.head(3)

,Rank,Overall,Home,Road,Eastern Conference,Western Conference,Atlantic Div,Central Div,Southeast Div,Northwest Div,...,Post All-Star,Margin ≤3,Margin ≥10,Oct,Nov,Dec,Jan,Feb,Mar,Apr
Team,,,,,,,,,,,,,,,,,,,,,
Miami Heat,1,66-16,37-4,29-12,41-11,25-5,14-4,12-6,15-1,8-2,...,30-2,9-3,39-8,1-0,10-3,10-5,8-5,12-1,17-1,8-1
Oklahoma City Thunder,2,60-22,34-7,26-15,21-9,39-13,7-3,8-2,6-4,10-6,...,21-8,3-6,44-6,NaN,13-4,11-2,11-5,7-4,12-5,6-2
San Antonio Spurs,3,58-24,35-6,23-18,25-5,33-19,8-2,9-1,8-2,9-9,...,16-12,9-5,31-10,1-0,12-4,12-4,12-3,8-3,10-4,3-6


In [26]:
# Create a new feature -> Home Team Ranks Higher
# First create a function that iterates through the df to determine if the home team has a higher rank based on rank df

def home_team_ranks_higher(row):
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    
    # Adjusting the New Orleans team names due to off-season league changes between 12-13 & 13-14
    if home_team == "New Orleans Pelicans":
        home_team = "New Orleans Hornets"
    if visitor_team == "New Orleans Pelicans":
        visitor_team = "New Orleans Hornets"
    
    home_rank = rank.loc[home_team]["Rank"]
    visitor_rank = rank.loc[visitor_team]["Rank"]
    
    return home_rank < visitor_rank   # The higher ranking will be the lower number

In [27]:
# Create new column by using .apply(home_team_ranks_higher) across the DataFrame
df["Home Team Ranks Higher"] = df.apply(home_team_ranks_higher, axis = 1)

# Display the selected columns on the sliced DataFrame (first 5 rows)
df[:5][['Date', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 'Home Win', 'Home Team Ranks Higher']]

,Date,Visitor Team,Visitor Score,Home Team,Home Score,Home Win,Home Team Ranks Higher
0,2013-10-29,Orlando Magic,87,Indiana Pacers,97,True,True
1,2013-10-29,Los Angeles Clippers,103,Los Angeles Lakers,116,True,False
2,2013-10-29,Chicago Bulls,95,Miami Heat,107,True,True
3,2013-10-30,Brooklyn Nets,94,Cleveland Cavaliers,98,True,False
4,2013-10-30,Atlanta Hawks,109,Dallas Mavericks,118,True,False


______________________________________________________________________________________________________________________

In [ ]:
# # Add new column without using a function & .apply
# df["Home Team Ranks Higher"] = 0

# for index, row in df.iterrows():
#     home_team = row["Home Team"]
#     visitor_team = row["Visitor Team"]
   
#     # Adjusting the New Orleans team names due to off-season league changes between 12-13 & 13-14
#     if home_team == "New Orleans Pelicans":
#         home_team = "New Orleans Hornets"
#     elif visitor_team == "New Orleans Pelicans":
#         visitor_team = "New Orleans Hornets"
    
#     home_rank = rank[rank["Team"] == home_team]["Rank"].values[0]
#     visitor_rank = rank[rank["Team"] == visitor_team]["Rank"].values[0]
#     row["Home Team Ranks Higher"] = int(home_rank > visitor_rank)
#     df.loc[index] = row

In [ ]:
# df[:5][['Date', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 'Home Win']]

______________________________________________________________________________________________________________________

In [28]:
# Decision Tree Classifier based on if Home Team has a Higher Ranking

# Use selected features as input for the classifier (target)
X_homehigher = df[["Home Last Win", "Visitor Last Win", "Home Team Ranks Higher"]].values

# Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_homehigher, y_true, scoring = scorer)

# Print results
print("Using whether the home team is ranked higher")
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
F1: 59.4525%


When taking into consideration whether the home team has a higher rank in the previous season's standings, the model's results show 59.45%


*** This result is .2 higher than the lecture ***

In [29]:
# Adjusting the parameters of the algorithm using GridSearchCV to test if there is an improvement in the model's score

from sklearn.grid_search import GridSearchCV

parameter_space = {
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
}
dtc = DecisionTreeClassifier(random_state = 14)

grid = GridSearchCV(dtc, parameter_space, scoring = scorer)
grid.fit(X_homehigher, y_true)

print('F1: {0:.4f}%'.format(grid.best_score_ * 100))

F1: 59.4525%


/Users/slic_vic/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


By adjusting the parameters (tuning) of the Decision Tree, there is no change in the models

## Decision Tree Classifier: Based on whether the Home Team Won the Last Matchup

______________________________________________________________________________________________________________________

*** This function was entered exactly, but when applied, there is an error message. Used the inefficient method to continue with the lecture/notebook ***

*** Skip to 22:20 in video ***

In [ ]:
# Build a function that determines whether a team won the last matchup between the 2 teams
# This does not take into consideration the home/visitor teams

last_game_winner = defaultdict(int)

def home_team_won_last(row):
    home_team = row['Home Team']
    visitor_team = row['Visistor Team']
    
    # Sort for a consistent ordering
    teams = tuple(sorted([home_team, visitor_team]))
    # Parse the row for which team won the last matchup, then add a 1 if the Home Team won
    result = 1 if last_game_winner[teams] == row['Home Team'] else 0
    
    # Update record for next matchup
    winner = row['Home Team'] if row['Home Win'] else row['Visitor Team']
    last_game_winner[teams] = winner
    
    return result

# Function works, but is not applying across DataFrame (see cell below)

In [ ]:
# Create new column by using .apply(home_team_won_last) across the DataFrame
# df['Home Team Won Last'] = df.apply(home_team_won_last, axis = 1)

In [ ]:
# Display the selected columns on the sliced DataFrame (5 random rows)
# df[90:100][['Date', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 'Home Win', 'Home Team Won Last']]

______________________________________________________________________________________________________________________

In [30]:
# Add a new feature without using function -> Determines whether a team won the last matchup between the 2 teams
# This does not take into consideration the home/visitor teams

last_game_winner = defaultdict(int)
df["Home Team Won Last"] = 0

for index, row in df.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    teams = tuple(sorted([home_team, visitor_team]))  # Sort for a consistent ordering
    
    # Set in the row, who won the last encounter
    row["Home Team Won Last"] = 1 if last_game_winner[teams] == row["Home Team"] else 0
    df.loc[index] = row
    
    # Who won this one?
    winner = row["Home Team"] if row["Home Win"] else row["Visitor Team"]
    last_game_winner[teams] = winner

In [31]:
# Display the selected columns on the sliced DataFrame (10 random rows)
df[90:100][['Date', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 'Home Win', 'Home Team Won Last']]

,Date,Visitor Team,Visitor Score,Home Team,Home Score,Home Win,Home Team Won Last
90,2013-11-09,Dallas Mavericks,91,Milwaukee Bucks,83,False,0
91,2013-11-09,Portland Trail Blazers,96,Sacramento Kings,85,False,0
92,2013-11-09,Utah Jazz,91,Toronto Raptors,115,True,0
93,2013-11-10,Minnesota Timberwolves,113,Los Angeles Lakers,90,False,0
94,2013-11-10,San Antonio Spurs,120,New York Knicks,89,False,0
95,2013-11-10,Washington Wizards,105,Oklahoma City Thunder,106,True,0
96,2013-11-10,New Orleans Pelicans,94,Phoenix Suns,101,True,1
97,2013-11-11,Orlando Magic,105,Boston Celtics,120,True,1
98,2013-11-11,Atlanta Hawks,103,Charlotte Bobcats,94,False,0
99,2013-11-11,Cleveland Cavaliers,81,Chicago Bulls,96,True,0


In [32]:
# Use selected features as input for the classifier (target)
X_home_higher = df[['Home Last Win', 'Visitor Last Win', "Home Team Ranks Higher", "Home Team Won Last"]].values

# Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_home_higher, y_true, scoring = scorer)

# Print results
print("Using whether the home team won the last matchup")
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

Using whether the home team won the last matchup
F1: 60.1726%


After adding the feature that takes into account whether the home team won the last matchup between the teams, the model's accuracy increased to 60.17%

*** This is about .09 lower than the lecture ***

### Decision Tree Classifier: Encoding Teams as Features

In [33]:
df['Home Team'].values

array(['Indiana Pacers', 'Los Angeles Lakers', 'Miami Heat', ...,
       'Portland Trail Blazers', 'Sacramento Kings', 'San Antonio Spurs'],
      dtype=object)

In [34]:
# Use LabelEncoder to transform the teams by assigning numbers to them
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

encoding = LabelEncoder()
encoding.fit(df["Home Team"].values)

LabelEncoder()

In [35]:
# Numerical values assigned to the home and visitor teams
home_teams = encoding.transform(df["Home Team"].values)
visitor_teams = encoding.transform(df["Visitor Team"].values)

# Stacks arrays in a sequence vertically (row-wise) & transposes the data
X_teams = np.vstack([home_teams, visitor_teams]).T

In [36]:
# OneHotEncoder takes each of the features of the numerical valued-teams & identify if it was the home or visitor team
# Will display a 1 if True, 0 if False
onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).todense()

In [37]:
# View the data size of X_teams
X_teams.shape

(1230, 60)

In [38]:
# Print the first 30 elements in X_teams
print('Home:', X_teams[0:30])
print('Visitor:', X_teams[0,30:])

Home: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
Visitor: [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0.]]


In [39]:
# Run the classifier on the new teams as features
# Decision Tree Classifier
dtc = DecisionTreeClassifier(random_state = 14)
scores = cross_val_score(dtc, X_teams, y_true, scoring = scorer)

# Print results
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

F1: 59.9439%


When encoding the teams as individual features, the accuracy of the model resulted in an F1 score of 59.94%.

*** This result is .32 higher than the lecture ***

## Random Forest Classifiers

### Random Forest Classifier: Use ONLY the Teams Encoded as Features

In [40]:
# Import Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier
rfc = RandomForestClassifier(random_state = 14)
scores = cross_val_score(rfc, X_teams, y_true, scoring = scorer)

# Print results
print("Using full team labels is ranked higher")
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

Using full team labels is ranked higher
F1: 60.8530%


*** This result is .4 higher than the lecture ***

### Random Forest Classifier: Adjusting Parameters with GridSearchCV for Encoded Teams

In [41]:
# Set up more detailed parameters for the Random Forest Classifier in an attempt to improve accuracy score
# Adjusting the parameters of the algorithm using GridSearchCV

parameter_space = {
                   "max_features": [2, 10, 50, 'auto'],
                   "n_estimators": [50, 100, 200],
                   "criterion": ["gini", "entropy"],
                   "min_samples_leaf": [1, 2, 4, 6],
                   }

# Random Forest Classifier
rfc = RandomForestClassifier(random_state = 14)
grid = GridSearchCV(rfc, parameter_space, scoring = scorer)
grid.fit(X_teams, y_true)

# Print results
print("F1: {0:.4f}%".format(grid.best_score_ * 100))
print(grid.best_estimator_)

F1: 64.3611%
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=14, verbose=0, warm_start=False)


After adjusting the parameters when using the encoded teams as features, the model's results improved to 64.36%

### Random Forest Classifier: Use ALL Features for Data

In [42]:
# Combine the team features (from one hot endoder) with the other previously determined features
X_all = np.hstack([X_home_higher, X_teams])
print(X_all.shape)

(1230, 64)


In [43]:
# Random Forest Classifier
rfc = RandomForestClassifier(random_state = 14)
scores = cross_val_score(rfc, X_all, y_true, scoring = scorer)

# Print results
print("F1: {0:.4f}%".format(np.mean(scores) * 100))

F1: 61.4918%


There was a slight improvement to 61.49% with the additional features used in addition to the team features

*** This result is 1.1 higher than the lecture results ***

In [44]:
# Set up more detailed parameters for the Random Forest Classifier in an attempt to improve accuracy score
# Adjusting the parameters of the algorithm using GridSearchCV

parameter_space = {
                   "max_features": [2, 10, 50, 'auto'],
                   "n_estimators": [50, 100, 200],
                   "criterion": ["gini", "entropy"],
                   "min_samples_leaf": [1, 2, 4, 6],
                   }

# Random Forest Classifier
rfc = RandomForestClassifier(random_state = 14)
grid = GridSearchCV(rfc, parameter_space, scoring = scorer)
grid.fit(X_all, y_true)

# Print results
print("F1: {0:.4f}%".format(grid.best_score_ * 100))
print(grid.best_estimator_)

F1: 62.9018%
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=14, verbose=0, warm_start=False)


After modifying the parameters of the Random Forest Classifier and using all of the features, the result was 62.9%. While this wasn't the best performing RFC, it did perform better than the initial baseline and the majority of the Decision Tree Classifiers.

*** This result is 1 lower than the lecture results ***

*** INSERT GRAPHS COMPARING MODEL SCORE RESULTS ***

# Testing on 2014-2015 Season

In [45]:
# Load the data for the 2014-15 season
df_15 = pd.read_csv('./datasets/NBA Regular Season Results 2014-2015.csv', parse_dates = ['Date'], 
                    dtype = {'Attendance':int}) # Attendence is still showing up as a float (HOW TO FIX?)

# Correct column names
df_15.columns = ['Date', 'Start Time (ET)', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 'Box Score', 
              'OT', 'Attendance', 'Notes']

df_15.head(7)

,Date,Start Time (ET),Visitor Team,Visitor Score,Home Team,Home Score,Box Score,OT,Attendance,Notes
0,2014-10-28,10:30 pm,Houston Rockets,108,Los Angeles Lakers,90,Box Score,NaN,18997.0,NaN
1,2014-10-28,8:00 pm,Orlando Magic,84,New Orleans Pelicans,101,Box Score,NaN,17097.0,NaN
2,2014-10-28,8:00 pm,Dallas Mavericks,100,San Antonio Spurs,101,Box Score,NaN,19615.0,NaN
3,2014-10-29,7:30 pm,Brooklyn Nets,105,Boston Celtics,121,Box Score,NaN,18624.0,NaN
4,2014-10-29,7:00 pm,Milwaukee Bucks,106,Charlotte Hornets,108,Box Score,OT,19439.0,NaN
5,2014-10-29,9:00 pm,Detroit Pistons,79,Denver Nuggets,89,Box Score,NaN,17136.0,NaN
6,2014-10-29,7:00 pm,Philadelphia 76ers,91,Indiana Pacers,103,Box Score,NaN,18165.0,NaN


In [46]:
# Load the previous season's standings
rank_14 = pd.read_csv('./datasets/2013-2014 Regular Season Standings.csv', index_col = 'Team')

# Rename columns in the rank DataFrame
rank_14.columns = ['Rank', 'Overall', 'Home', 'Road', 'Eastern Conference', 'Western Conference', 
                'Atlantic Div', 'Central Div', 'Southeast Div', 'Northwest Div', 'Pacific Div', 'Southwest Div', 
                'Pre All-Star', 'Post All-Star', 'Margin ≤3', 'Margin ≥10', 'Oct', 'Nov', 'Dec', 'Jan', 'Feb', 
                'Mar', 'Apr']

rank_14.head()

,Rank,Overall,Home,Road,Eastern Conference,Western Conference,Atlantic Div,Central Div,Southeast Div,Northwest Div,...,Post All-Star,Margin ≤3,Margin ≥10,Oct,Nov,Dec,Jan,Feb,Mar,Apr
Team,,,,,,,,,,,,,,,,,,,,,
San Antonio Spurs,1,62-20,32-9,30-11,24-6,38-14,8-2,7-3,9-1,11-7,...,24-5,7-0,35-12,1-0,13-3,11-4,8-6,9-3,16-0,4-4
Oklahoma City Thunder,2,59-23,34-7,25-16,23-7,36-16,8-2,8-2,7-3,11-5,...,16-11,10-4,33-8,1-0,10-3,14-3,13-4,6-5,10-4,5-4
Los Angeles Clippers,3,57-25,34-7,23-18,21-9,36-16,9-1,7-3,5-5,12-6,...,20-7,4-3,30-9,1-1,11-4,9-7,12-4,7-4,13-2,4-3
Indiana Pacers,4,56-26,35-6,21-20,38-14,18-12,15-3,12-4,11-7,6-4,...,16-14,4-3,29-17,2-0,13-1,10-4,10-5,9-3,8-10,4-3
Houston Rockets,5,54-28,33-8,21-20,23-7,31-21,7-3,8-2,8-2,11-7,...,18-11,7-6,30-9,1-0,12-5,8-8,10-4,8-2,10-4,5-5
Miami Heat,6,54-28,32-9,22-19,34-18,20-10,9-9,13-5,12-4,6-4,...,17-14,8-7,29-10,1-1,12-2,11-4,8-6,9-1,10-8,3-6
Portland Trail Blazers,7,54-28,31-10,23-18,23-7,31-21,9-1,9-1,5-5,13-3,...,18-11,8-9,24-13,0-1,13-2,12-4,8-6,7-5,8-9,6-1
Golden State Warriors,8,51-31,27-14,24-17,20-10,31-21,7-3,7-3,6-4,11-7,...,20-9,11-8,28-9,1-1,8-7,11-5,9-6,7-4,9-5,6-3
Memphis Grizzlies,9,50-32,27-14,23-18,21-9,29-23,6-4,7-3,8-2,11-7,...,21-9,10-4,18-13,0-1,8-7,5-9,12-3,7-5,12-5,6-2


Add the features that were added to the testing data: 
- Home Win 
- Home Last Win
- Visitor Last Win
- Home Win Streak
- Visitor Win Streak
- Home Team Ranks Higher
- Home Team Won Last

**Home Win**

In [47]:
# Determine whether the home team won
df_15['Home Win'] = df_15['Visitor Score'] < df_15['Home Score']

# Assign "class values" to the Home Win column
y_true = df_15['Home Win'].values

In [48]:
# Review new column with correctly populated fields
df.head()

,Date,Start Time (ET),Visitor Team,Visitor Score,Home Team,Home Score,Box Score,OT,Attendance,Notes,Home Win,Home Last Win,Visitor Last Win,Home Win Streak,Visitor Win Streak,Home Team Ranks Higher,Home Team Won Last
0,2013-10-29,7:00 pm,Orlando Magic,87,Indiana Pacers,97,Box Score,NaN,18165,NaN,True,False,False,0,0,True,0
1,2013-10-29,10:30 pm,Los Angeles Clippers,103,Los Angeles Lakers,116,Box Score,NaN,18997,NaN,True,False,False,0,0,False,0
2,2013-10-29,8:00 pm,Chicago Bulls,95,Miami Heat,107,Box Score,NaN,19964,NaN,True,False,False,0,0,True,0
3,2013-10-30,7:00 pm,Brooklyn Nets,94,Cleveland Cavaliers,98,Box Score,NaN,20562,NaN,True,False,False,0,0,False,0
4,2013-10-30,8:30 pm,Atlanta Hawks,109,Dallas Mavericks,118,Box Score,NaN,19834,NaN,True,False,False,0,0,False,0


**Home Last Win & Visitor Last Win**

In [49]:
# Create two new columns, with all of the values set to False
df_15['Home Last Win'] = False
df_15['Visitor Last Win'] = False

# Determining whether the home and visitor teams won their last games
# This will update the Home Last Win & Visitor Last Win columns to either remain False or change to True

from collections import defaultdict

won_last = defaultdict(int)   # The default value of int is 0

# Inefficient method to deterime previous wins
for index, row in df_15.iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    row['Home Last Win'] = won_last[home_team]
    row['Visitor Last Win'] = won_last[visitor_team]
    # Set current win
    won_last[home_team] = row['Home Win']
    won_last[visitor_team] = not row['Home Win']
df_15.sample(10)

,Date,Start Time (ET),Visitor Team,Visitor Score,Home Team,Home Score,Box Score,OT,Attendance,Notes,Home Win,Home Last Win,Visitor Last Win
542,2015-01-09,8:30 pm,Minnesota Timberwolves,84,Milwaukee Bucks,98,Box Score,NaN,15480.0,NaN,True,False,False
474,2014-12-31,1:00 pm,Sacramento Kings,84,Boston Celtics,106,Box Score,NaN,18624.0,NaN,True,False,False
232,2014-11-28,8:30 pm,Sacramento Kings,104,San Antonio Spurs,112,Box Score,NaN,18581.0,NaN,True,False,False
1222,2015-04-15,10:30 pm,Sacramento Kings,122,Los Angeles Lakers,99,Box Score,NaN,18997.0,NaN,False,False,False
1116,2015-04-01,7:30 pm,Brooklyn Nets,100,New York Knicks,98,Box Score,NaN,19812.0,NaN,False,False,False
1175,2015-04-10,7:30 pm,Washington Wizards,80,Brooklyn Nets,117,Box Score,NaN,17732.0,NaN,True,False,False
1053,2015-03-24,7:30 pm,Toronto Raptors,104,Detroit Pistons,108,Box Score,NaN,14420.0,NaN,True,False,False
708,2015-01-31,8:30 pm,Portland Trail Blazers,88,Milwaukee Bucks,95,Box Score,NaN,18717.0,NaN,True,False,False
606,2015-01-17,10:00 pm,Los Angeles Clippers,117,Sacramento Kings,108,Box Score,NaN,16601.0,NaN,False,False,False
950,2015-03-11,7:30 pm,Memphis Grizzlies,92,Boston Celtics,95,Box Score,NaN,17135.0,NaN,True,False,False


**Home Win Streak & Visitor Win Streak**

In [50]:
# Creating columns for winning streaks
df_15["Home Win Streak"] = 0
df_15["Visitor Win Streak"] = 0

# Did the home and visitor teams win their last game?

from collections import defaultdict

win_streak = defaultdict(int)

for index, row in df_15.iterrows():  # Note that this is not the most efficient method
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["Home Win Streak"] = win_streak[home_team]
    row["Visitor Win Streak"] = win_streak[visitor_team]
    df_15.loc[index] = row    
    
    # Set current win streak
    if row["Home Win"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1
        
df_15.sample(10)

,Date,Start Time (ET),Visitor Team,Visitor Score,Home Team,Home Score,Box Score,OT,Attendance,Notes,Home Win,Home Last Win,Visitor Last Win,Home Win Streak,Visitor Win Streak
1069,2015-03-25,9:00 pm,Portland Trail Blazers,92,Utah Jazz,89,Box Score,NaN,19911.0,NaN,False,False,False,0,0
630,2015-01-21,8:00 pm,Dallas Mavericks,98,Minnesota Timberwolves,75,Box Score,NaN,13737.0,NaN,False,False,False,0,2
1092,2015-03-29,4:30 pm,Philadelphia 76ers,86,Cleveland Cavaliers,87,Box Score,NaN,20562.0,NaN,True,False,False,0,0
546,2015-01-09,8:30 pm,Phoenix Suns,95,San Antonio Spurs,100,Box Score,NaN,18581.0,NaN,True,False,False,0,4
517,2015-01-05,10:00 pm,Los Angeles Lakers,94,Portland Trail Blazers,98,Box Score,NaN,19827.0,NaN,True,False,False,0,1
13,2014-10-29,7:30 pm,Atlanta Hawks,102,Toronto Raptors,109,Box Score,NaN,19800.0,NaN,True,False,False,0,0
1134,2015-04-03,7:00 pm,New York Knicks,87,Washington Wizards,101,Box Score,NaN,19389.0,NaN,True,False,False,1,0
17,2014-10-30,10:30 pm,Oklahoma City Thunder,90,Los Angeles Clippers,93,Box Score,NaN,19060.0,NaN,True,False,False,0,0
595,2015-01-16,8:30 pm,Portland Trail Blazers,96,San Antonio Spurs,110,Box Score,NaN,18581.0,NaN,True,False,False,1,0
485,2015-01-02,10:30 pm,Toronto Raptors,105,Golden State Warriors,126,Box Score,NaN,19596.0,NaN,True,False,False,2,0


______________________________________________________________________________________________________________________

**Home Team Ranks Higher**

In [51]:
# Build function to add Home Team Ranks Higher feature
def home_team_ranks_higher_2015(row):
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    
    # Adjusting the Charlotte team names due to off-season league changes between 13-14 & 14-15
    if home_team == "Charlotte Hornets":
        home_team = "Charlotte Bobcats"
    if visitor_team == "Charlotte Hornets":
        visitor_team = "Charlotte Bobcats"
    
    home_rank = rank_14.loc[home_team]['Rank']
    visitor_rank = rank_14.loc[visitor_team]['Rank']
    
    # The higher ranking will be the lower number
    return home_rank < visitor_rank

In [52]:
# Create new column by using .apply(home_team_ranks_higher_2015) across the DataFrame
df_15["Home Team Ranks Higher"] = df_15.apply(home_team_ranks_higher_2015, axis = 1)
df_15.head()

,Date,Start Time (ET),Visitor Team,Visitor Score,Home Team,Home Score,Box Score,OT,Attendance,Notes,Home Win,Home Last Win,Visitor Last Win,Home Win Streak,Visitor Win Streak,Home Team Ranks Higher
0,2014-10-28,10:30 pm,Houston Rockets,108,Los Angeles Lakers,90,Box Score,NaN,18997.0,NaN,False,False,False,0,0,False
1,2014-10-28,8:00 pm,Orlando Magic,84,New Orleans Pelicans,101,Box Score,NaN,17097.0,NaN,True,False,False,0,0,True
2,2014-10-28,8:00 pm,Dallas Mavericks,100,San Antonio Spurs,101,Box Score,NaN,19615.0,NaN,True,False,False,0,0,True
3,2014-10-29,7:30 pm,Brooklyn Nets,105,Boston Celtics,121,Box Score,NaN,18624.0,NaN,True,False,False,0,0,False
4,2014-10-29,7:00 pm,Milwaukee Bucks,106,Charlotte Hornets,108,Box Score,OT,19439.0,NaN,True,False,False,0,0,True


In [ ]:
# Add new column without using a function & .apply
# df_15["Home Team Ranks Higher"] = 0

# for index, row in df_15.iterrows():
#     home_team = row["Home Team"]
#     visitor_team = row["Visitor Team"]
   
    # Adjusting the New Orleans team names due to off-season league changes between 12-13 & 13-14
#     if home_team == "Charlotte Hornets":
#         home_team = "Charlotte Bobcats"
#     elif visitor_team == "Charlotte Hornets":
#         visitor_team = "Charlotte Bobcats"
    
#     home_rank = rank_14[rank_14["Team"] == home_team]["Rank"].values[0]
#     visitor_rank = rank_14[rank_14["Team"] == visitor_team]["Rank"].values[0]
#     row["Home Team Ranks Higher"] = int(home_rank > visitor_rank)
#     df_15.loc[index] = row

______________________________________________________________________________________________________________________

**Home Team Won Last**

In [54]:
# Add a new feature without using function -> Determines whether a team won the last matchup between the 2 teams
# This does not take into consideration the home/visitor teams

last_match_winner = defaultdict(int)
df_15["Home Team Won Last"] = 0

for index, row in df_15.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    teams = tuple(sorted([home_team, visitor_team]))  # Sort for a consistent ordering
    
    # Set in the row, who won the last encounter
    row["Home Team Won Last"] = 1 if last_match_winner[teams] == row["Home Team"] else 0
    df_15.loc[index] = row
    
    # Who won this one?
    winner = row["Home Team"] if row["Home Win"] else row["Visitor Team"]
    last_match_winner[teams] = winner
    
df_15.sample(7)

,Date,Start Time (ET),Visitor Team,Visitor Score,Home Team,Home Score,Box Score,OT,Attendance,Notes,Home Win,Home Last Win,Visitor Last Win,Home Win Streak,Visitor Win Streak,Home Team Ranks Higher,Home Team Won Last
1198,2015-04-12,3:00 pm,Brooklyn Nets,73,Milwaukee Bucks,96,Box Score,NaN,16504.0,NaN,True,False,False,1,1,False,0
1190,2015-04-11,7:00 pm,New York Knicks,80,Orlando Magic,79,Box Score,NaN,17207.0,NaN,False,False,False,0,0,False,1
395,2014-12-20,8:00 pm,Atlanta Hawks,104,Houston Rockets,97,Box Score,NaN,16998.0,NaN,False,False,False,0,2,True,0
696,2015-01-30,7:30 pm,Sacramento Kings,90,Cleveland Cavaliers,101,Box Score,NaN,20562.0,NaN,True,False,False,8,0,True,0
1,2014-10-28,8:00 pm,Orlando Magic,84,New Orleans Pelicans,101,Box Score,NaN,17097.0,NaN,True,False,False,0,0,True,0
364,2014-12-16,8:00 pm,Utah Jazz,111,New Orleans Pelicans,119,Box Score,NaN,13179.0,NaN,True,False,False,0,0,True,1
205,2014-11-25,7:30 pm,Golden State Warriors,114,Miami Heat,97,Box Score,NaN,19647.0,NaN,False,False,False,2,5,True,0


In [55]:
# Review the added features in the 2014-2015 season DataFrame
df_15[['Date', 'Visitor Team', 'Visitor Score', 'Home Team', 'Home Score', 'Home Win', 'Home Last Win', 
       'Visitor Last Win', 'Home Team Ranks Higher', 'Home Win Streak', 'Visitor Win Streak', 
       'Home Team Won Last']][:5]

,Date,Visitor Team,Visitor Score,Home Team,Home Score,Home Win,Home Last Win,Visitor Last Win,Home Team Ranks Higher,Home Win Streak,Visitor Win Streak,Home Team Won Last
0,2014-10-28,Houston Rockets,108,Los Angeles Lakers,90,False,False,False,False,0,0,0
1,2014-10-28,Orlando Magic,84,New Orleans Pelicans,101,True,False,False,True,0,0,0
2,2014-10-28,Dallas Mavericks,100,San Antonio Spurs,101,True,False,False,True,0,0,0
3,2014-10-29,Brooklyn Nets,105,Boston Celtics,121,True,False,False,False,0,0,0
4,2014-10-29,Milwaukee Bucks,106,Charlotte Hornets,108,True,False,False,True,0,0,0


In [57]:
# Use selected features as input for the classifier (target variable) for if the home team is ranked higher
X_home_higher_15 = df_15[['Home Last Win', 'Visitor Last Win', 'Home Team Ranks Higher', 'Home Team Won Last']].values

In [ ]:
# Reuse the trained team encoding and onehot classifier
# encoding = LabelEncoder()
# encoding.fit(df['Home Team].values)

home_teams_15 = encoding.transform(df_15['Home Team'].values)
visitor_teams_15 = encoding.transform(df_15['Visitor Team'].values)

X_teams_15 = np.vstack([home_teams_15, visitor_teams_15]).T
X_teams_15 = onehot.transform(X_teams_15).todense()

# There was a team name change in the test set of data, causing the previous encoder to not carry over. HOW TO FIX?

In [ ]:
X_all_15 = np.hstack([X_home_higher_15, X_teams_15])

______________________________________________________________________________________________________________________